# Setting up a PEST interface from MODFLOW6 using the `PstFrom` class and `pypestutils` using a quadtree MF6 model

In this notebook, we will combine the power of `pypestutils` with the power of `PstFrom`.  Then we will also demonstrate a technique to condition the prior parameter ensemble to imprecise/uncertain "direct" parameter data. Hold on tight!

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

This notebook uses `flopy`, `pyEMU`, `modflow6` and `pestpp-ies`.  Let's make sure we can import those deps:

In [ ]:
import pyemu
import flopy

And we can use methods `get_modflow()` and `get_pestpp()`:

In [ ]:
bin_dir = "quadtree_bin"
if os.path.exists(bin_dir):
   shutil.rmtree(bin_dir)
os.makedirs(bin_dir)
flopy.utils.get_modflow(bin_dir,downloads_dir=bin_dir)
pyemu.utils.get_pestpp(bin_dir,downloads_dir=bin_dir)

An existing MODFLOW6 model is in the directory `freyberg_mf6`.  Lets check it out:

In [ ]:
org_model_ws = os.path.join('freyberg_quadtree')
os.listdir(org_model_ws)

You can see that all the input array and list data for this model have been written "externally" - this is key to using the `PstFrom` class. 

In [ ]:
id_arr = np.loadtxt(os.path.join(org_model_ws,"freyberg6.disv_idomain_layer3.txt"))

Now let's copy those files to a temporary location just to make sure we don't goof up those original files:

In [ ]:
tmp_model_ws = "temp_ppu_quadtree"
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
for bin_file in os.listdir(bin_dir):
    shutil.copy2(os.path.join(bin_dir,bin_file),os.path.join(tmp_model_ws,bin_file))
os.listdir(tmp_model_ws)


Now we need just a tiny bit of info about the spatial discretization of the model - this is needed to work out separation distances between parameters for build a geostatistical prior covariance matrix later.

Here we will load the flopy sim and model instance just to help us define some quantities later - flopy is not required to use the `PstFrom` class.

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=tmp_model_ws)
m = sim.get_model("freyberg6")
m.modelgrid.set_coord_info(angrot=-55) #for some reason flopy isnt picking up angrot from the disv options...


In [ ]:
m.dis.top.plot()

In [ ]:
top_arr = m.dis.top.array

Now we can instantiate a `PstFrom` class instance

In [ ]:
template_ws = "freyberg6_unstruct_template"
pf = pyemu.utils.PstFrom(original_d=tmp_model_ws, new_d=template_ws,
                 remove_existing=True,
                 longnames=True,
                 zero_based=False,start_datetime="1-1-2018")

## Observations

So now that we have a `PstFrom` instance, but its just an empty container at this point, so we need to add some PEST interface "observations" and "parameters".  Let's start with observations using MODFLOW6 head.  These are stored in `heads.csv`.  Note the zero-based layer-row-column name is stored in the site names:

In [ ]:
hdsdf = pd.read_csv(os.path.join(tmp_model_ws,"heads.csv"),index_col=0)
hdsdf

The main entry point for adding observations is (surprise) `PstFrom.add_observations()`.  This method works on the list-type observation output file.  We need to tell it what column is the index column (can be string if there is a header or int if no header) and then what columns contain quantities we want to monitor (e.g. "observe") in the control file - in this case we want to monitor all columns except the index column:

In [ ]:
hds_df = pf.add_observations("heads.csv",insfile="heads.csv.ins",index_cols="time",
                    use_cols=list(hdsdf.columns.values),prefix="hds",)
hds_df

We can see that it returned a dataframe with lots of useful info: the observation names that were formed (`obsnme`), the values that were read from `heads.csv` (`obsval`) and also some generic weights and group names.  At this point, no control file has been created, we have simply prepared to add this observations to the control file later.  

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".ins")]

Nice!  We also have a PEST-style instruction file for those obs.

Now lets do the same for SFR observations:

In [ ]:
df = pd.read_csv(os.path.join(tmp_model_ws, "sfr.csv"), index_col=0)
sfr_df = pf.add_observations("sfr.csv", insfile="sfr.csv.ins", index_cols="time", use_cols=list(df.columns.values))
sfr_df

Sweet as!  Now that we have some observations, let's add parameters!

## Parameters

In the `PstFrom` realm, all parameters are setup as multipliers against existing array and list files.  This is a good thing because it lets us preserve the existing model inputs and treat them as the mean of the prior parameter distribution. It also let's us use mixtures of spatial and temporal scales in the parameters to account for varying scale of uncertainty. 

Since we are all sophisticated and recognize the importance of expressing spatial and temporal uncertainty (e.g. heterogeneity) in the model inputs (and the corresponding spatial correlation in those uncertain inputs), let's use geostatistics to express uncertainty.  But lets use the awesome sauce in `PyPestUtils` to do fancy things with our pilot point parameters.  To start, we need pilot point location information.  We can generate this ourselves however we want - we need spatial info (ie "x" and "y" coordinates) and we need at least default values for "value" and "zone".  If we want to have spatially-varying geostatistical hyper parameters, then we also need columns for "bearing","aniso", and "corrlen", which are the bearing (duh), anisotropy ratio and correlation length at each pilot point location.  Like we said, you can generate this however works best for you - maybe in a GIS or with geopandas, whatevs.  For this notebook, we will use the same (or similar) pilot point info that was used in the structured freyberg `pypestutils` notebook:

In [ ]:
from pypestutils import helpers as ppu

In [ ]:
ppdf = pd.read_csv(os.path.join("freyberg_aux_files","pp_info.csv"),index_col=0)
ppdf

In [ ]:
fig,ax = plt.subplots(1,1)
ax.set_aspect("equal")
#ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,top_arr)
#ax = m.dis.top.plot()

ax.scatter(m.modelgrid.xcellcenters,m.modelgrid.ycellcenters,alpha=0.5,color="0.5")
ax.scatter(ppdf.x,ppdf.y,marker=".",c="r")
plt.tight_layout()

In [ ]:
ppdf.loc[:,"value"] = 3.0  # HK in layer 1

Now lets visualize the various geostatistical hyper parameter pilot point attributes:

In [ ]:
id_mask = id_arr.copy()
id_mask[id_mask!=0] = np.nan
fig,axes = plt.subplots(1,3)
for ax,attr in zip(axes,["aniso","corrlen","bearing"]):
    #ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,id_mask)
    m.dis.top.plot(axes=[ax])
    ax.scatter(ppdf.x,ppdf.y,c=ppdf.loc[:,attr])
    ax.set_aspect("equal")
    ax.set_title(attr,loc="left")
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()

Sweet ez!  But how do we interpolate with this fancy-ness?  Well there is a helper for that.  To demo its use, lets gen up some rando "value" values...

In [ ]:
ppdf.loc[:,"value"] = np.random.normal(1.0,0.25,ppdf.shape[0])
results = ppu.interpolate_with_sva_pilotpoints_2d(ppdf,os.path.join(pf.new_d,"freyberg6.disv.grb"))

In [ ]:
for tag,arr in results.items():
    fig,ax = plt.subplots(1,1)
    ax.set_aspect("equal")
    ax.set_title(tag,loc="left")
    m.dis.top = arr
    ax = m.dis.top.plot(colorbar=True)
    
    #plt.colorbar(cb)
    plt.show()
    plt.close(fig)


Ok, thats pretty awesome.  So now lets proceed as usual with `PstFrom`

Now let's get the idomain array to use as a zone array - this keeps us from setting up parameters in inactive model cells:

In [ ]:
ib = m.dis.idomain[0].array

In [ ]:
hk_arr_files = [f for f in os.listdir(tmp_model_ws) if "npf_k_" in f and f.endswith(".txt")]
hk_arr_files

So those are the existing model input arrays for HK.  Notice we found the files in the temporary model workspace - `PstFrom` will copy all those files to the new model workspace for us in a bit...

Let's setup grid-scale multiplier parameters and fancy pilot point multipliers for HK in all layers.  Typically, we would want pass a `geostruct` to `PstFrom.add_parameters()` for the grid-scale parameters to that later when we generate the prior parameter ensemble, grid-scale parameter realizations would get some spatial correlation. But in this notebook, we are focused on using `pypestutils`, so when we get to the prior parameter ensemble generation, we will use `pypestutils` to generate spatially correlated grid-scale parameter realizations.

To start with, lets make sure to reset the "value" column in `ppdf` to 1.0 so we can use it as a multiplier...and...this where it gets deep: we need to have some spatial correlation on the pilot point spatial correlation attributes - yeesh!  This is where the term "hyper-parameter" comes from:  We are treating the controling attributes of the pilot point interpolation as "parameters" in the PEST sense, which means we want those attributes to have some spatial structure to them, otherwise, the resulting property fields will have implausible bearing, aniso, and/or correlation length variability (that is, those quantities will vary sharply over short distances, which we dont want). So we need to define, you guessed it, a geostatistical structure for...the...geostatistical structure?! #inception

In [ ]:
ppdf.loc[:,"value"] = 1.0
pp_v = pyemu.geostats.ExpVario(contribution=1.0, a=2000)
pp_gs = pyemu.geostats.GeoStruct(variograms=pp_v,transform="log")

Now, even though we are telling `PstFrom` that the pilot point parameters are "direct" type, we are actually going to use them as multipliers during the model run process (stay tuned for that part).  To do this, we need to track the model filenames and pilot point csv filenames

In [ ]:
pp_files,mod_files = [],[]
for hk_arr_file in hk_arr_files:
    layer = int(hk_arr_file.split(".")[1].split("layer")[1])
    base = hk_arr_file.split('.')[1].replace("_","")+"_attr:"
    pf.add_parameters(filenames=hk_arr_file,par_type="grid",
                       par_name_base=base+"gr",pargp=base+"gr",zone_array=np.atleast_2d(ib).transpose(),
                       upper_bound=2.,lower_bound=0.5,ult_ubound=100,ult_lbound=0.01)
    pppdf = ppdf.copy()
    
    pppdf.loc[:,"name"] = [n for n in pppdf.ppname.values]
    pppdf.loc[:,"ppname"] = pppdf.name.values
    pp_file = os.path.join(pf.new_d,base+"pp.csv")
    pppdf.to_csv(pp_file,index=False)
    pp_files.append(os.path.split(pp_file)[1])
    mod_files.append(hk_arr_file)
    df = pf.add_parameters(os.path.split(pp_file)[1],par_type="grid",index_cols={"ppname":"ppname","x":"x","y":"y"},
        use_cols=["value","bearing","aniso","corrlen"],
        par_name_base=[base+"pp",base+"bearing",base+"aniso",base+"corrlen"],
        pargp=[base+"pp",base+"bearing",base+"aniso",base+"corrlen"],
        upper_bound=[20,pppdf.bearing.max()*1.1,pppdf.aniso.max()*1.1,pppdf.corrlen.max()*1.1],
        lower_bound=[.05,pppdf.bearing.min()*.9,pppdf.aniso.min()*.9,pppdf.corrlen.min()*.9],
        par_style="direct",transform="log",geostruct=pp_gs)
    

Let's see what was created:

In [ ]:
tpl_files = [f for f in os.listdir(template_ws) if f.endswith(".tpl")]
tpl_files

In [ ]:
with open(os.path.join(template_ws,tpl_files[0]),'r') as f:
    for _ in range(4):
        print(f.readline().strip())
        

### Getting the `pypestutils` pilot point interpolation into the foward run process

So we added extra fancy pilot points to our pest interface with the smoothness of `PstFrom`.  But we still need to interpolate the pilot point values using the associated (estimated) hyper-parameters to the model grid at runtime.  To do this, we have setup a little helper funtion in a python script called "ppu_helpers.py". This helper function will read the ".grb" file at runtime so we need to make sure it always available. So let's copy the current ".grb" file to a file for safe keeping (in case MODFLOW-6 fails to run to competion and we lose the ".grb" file for the next run!)

In [ ]:
shutil.copy2(os.path.join(pf.new_d,"freyberg6.disv.grb"),os.path.join(pf.new_d,"org.grb"))

In [ ]:
_ = [print(line.rstrip()) for line in open("ppu_helpers.py",'r').readlines()]

So we just need to call this little function after the `PstFrom` runtime mult-to-model process happens so that we interpolate the pilot points to a grid-shaped array, then pickup the HK arrays that were created and multiply them by the interpolated array, and save for MODFLOW to see.  This is easy with `PstFrom`.  But first let's see if the function actually works:

In [ ]:
# save the pp file and model file info
df = pd.DataFrame({"model_file":mod_files,"pp_file":pp_files})
df.to_csv(os.path.join(pf.new_d,"pp_info.csv"))

In [ ]:
import ppu_helpers
ppu_helpers.setup_pps(pf.new_d)

In [ ]:
pf.add_py_function("ppu_helpers.py","apply_pps()",is_pre_cmd=True)

For added fun, lets track interpolated pp array and the log of the model input array as observations, just so we can see what its doing...(and we can also use these "observations" later for conditioning the parameter realizations to HK measurements)

In [ ]:
interp_files = [f for f in os.listdir(pf.new_d) if f.endswith(".txt") and f.startswith("interp")]
assert len(interp_files) == 3

In [ ]:
for interp_file in interp_files:
    base = interp_file.replace("_","").replace(".","").replace("txt","")
    pf.add_observations(interp_file,prefix=base,obsgp=base)

In [ ]:

log_files = [f for f in os.listdir(pf.new_d) if f.endswith(".txt") and f.startswith("log_")]
assert len(log_files) == 3


In [ ]:
for log_file in log_files:
    base = log_file.replace("_","").replace(".","").replace("txt","")
    pf.add_observations(log_file,prefix=base,obsgp=base)

### build the control file, pest interface files, and forward run script
At this point, we have some parameters and some observations, so we can create a control file...but first lets make sure we have set `mf6` as the model run command in `PstFrom`:

In [ ]:
# this conditional is just in case this block gets run multiple times...
if "mf6" not in pf.mod_sys_cmds:
    pf.mod_sys_cmds.append("mf6")
pst = pf.build_pst()

Let's see that magical forward run script that `PstFrom` writes:

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(template_ws,"forward_run.py"))]

## After building the control file

At this point, we can do some additional modifications that would typically be done that are problem specific.  Note that any modifications made after calling `PstFrom.build_pst()` will only exist in memory - you need to call `pf.pst.write()` to record these changes to the control file on disk.  Also note that if you call `PstFrom.build_pst()` after making some changes, these changes will be lost.  

Once you think you are happy with the initial interface design, the famous `noptmax=0` test is in order:

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d,"freyberg.pst"),version=2)
pyemu.os_utils.run("pestpp-ies freyberg.pst",cwd=pf.new_d)

# Generating a prior parameter ensemble

As usual, we will use `PstFrom`'s awesome `draw()` method to generate our prior parameter ensemble:

In [ ]:
num_reals = 50
pe = pf.draw(num_reals=num_reals,use_specsim=False)

## grid-scale parameter realizations with `pypestutils`

So that's kewl... but its important to notice that only the pilot-point-related parameter groups were drawn using a full covariance matrix.  This is because earlier, we did not pass a `geostruct` to `PstFrom.add_parameters()`.  Let's see how to generate grids-scale parameter realizations with `pypestutils` (note this approach is more general and support unstructed grids also...).  First using default arguments for the geostatistics:

In [ ]:
results = ppu.generate_2d_grid_realizations(os.path.join(pf.new_d,"org.grb"),num_reals=num_reals)

In [ ]:
#fig,ax = plt.subplots(1,1)
#ax.set_aspect("equal")
#ax.pcolormesh(m.modelgrid.xcellcenters,m.modelgrid.ycellcenters,results[0])
m.dis.top = results[0]
m.dis.top.plot()
plt.show()
plt.close(fig)

Lets use an off-grid angle bearing:

In [ ]:
results = ppu.generate_2d_grid_realizations(os.path.join(pf.new_d,"org.grb"),num_reals=num_reals,variobearing=90,varioaniso=3)
m.dis.top = results[0]
m.dis.top.plot(colorbar=True)
plt.show()
plt.close(fig)

## Mapping `pypestutils` realizations into the prior parameter ensemble

In [ ]:
par = pst.parameter_data
par.pargp.unique()

In [ ]:
grid_groups = ["npfklayer1_attr:gr","npfklayer2_attr:gr","npfklayer3_attr:gr"]

In [ ]:
for igrp,grp in enumerate(grid_groups):
    grpar = par.loc[par.pargp==grp,:].copy()
    assert grpar.shape[0] > 0
    grpar["i"] = grpar.i.astype(int)
    
    names,ivals,jvals = grpar.parnme.values,grpar.i.values,grpar.j.values
    results = ppu.generate_2d_grid_realizations(os.path.join(pf.new_d,"org.grb"),num_reals=num_reals,
                                            variobearing=1.0,varioaniso=1.0,variorange=1000,variance=0.0125,mean=0.0,random_seed=12345*igrp)
    print(grpar.shape)
    for ireal,real in enumerate(results):
        real = real[id_arr>0]
        #print(real.shape,ivals.shape,jvals.shape)
        pe.loc[pe.index[ireal],names] = 10**(real)
    print("group bound info: ",grpar.parlbnd.unique(),grpar.parubnd.unique())
    print("ensemble range info: ",pe.loc[:,names].values.min(),pe.loc[:,names].values.max())
    

Let's make sure it worked:

In [ ]:
arr = np.zeros_like(id_arr,dtype=float)
arr[id_arr>0] = pe.loc[pe.index[0],names].values
print(arr)
m.dis.top = arr
m.dis.top.plot(colorbar=True)

The range of the new realizations seems to jive with the parameter bounds (since we chose it that way...).  

So let's re-save the ensemble:

In [ ]:
pe.enforce()
pe.to_csv(os.path.join(template_ws,"prior.csv"))

Ok we made it!  This should have you in a good place to move forward with setting observation values and weights and running some stochastic unstructured mf6 runs!